# Setup

In [1]:
# import your standard packages
%run ../../../global_tools/start.py

# import your local functions
sys.path.insert(1, '../')
from local_functions import *

# make sure the figures plot inline rather than at the end
%matplotlib inline

Default libraries loaded.


# Parameters and Paths

In [2]:
inpath = "/tigress/GEOCLIM/LRGROUP/shared_data/woa18/"

outfn = '../../data/woa_processed.nc'

lat_bounds = [-1.5, 33]
lon_bounds = [48.5, 102.5]

# Get Data

In [3]:
for tt in tqdm(range(1,13)):
     
    if tt < 10:
        t='0'+str(tt)
    else:
        t=str(tt)
        
    if tt == 1:
        data=xr.open_dataset(inpath+"temp/1deg/monthly/woa18_decav_t"+t+"_01.nc", decode_times=False)
        
        depth = data.depth
        
        
        TEMP = np.full([12,data["t_an"].shape[1],data["t_an"].shape[2],data["t_an"].shape[3]],np.nan)
        DOXY = np.full([12,data["t_an"].shape[1],data["t_an"].shape[2],data["t_an"].shape[3]],np.nan)
        PSAL = np.full([12,data["t_an"].shape[1],data["t_an"].shape[2],data["t_an"].shape[3]],np.nan)
        NITRATE = np.full([12,data["t_an"].shape[1],data["t_an"].shape[2],data["t_an"].shape[3]],np.nan)
        PHOSPHATE = np.full([12,data["t_an"].shape[1],data["t_an"].shape[2],data["t_an"].shape[3]],np.nan)
       
        
    # TEMP
    data=xr.open_dataset(inpath+"temp/1deg/monthly/woa18_decav_t"+t+"_01.nc", decode_times=False)
    TEMP[tt-1,:,:,:] = np.array(data["t_an"][0,:,:,:])
    
    # PSAL
    data=xr.open_dataset(inpath+"salt/1deg/monthly/woa18_decav_s"+t+"_01.nc", decode_times=False)
    PSAL[tt-1,:,:,:] = np.array(data["s_an"][0,:,:,:])
    
    # DOXY
    data=xr.open_dataset(inpath+"oxygen/monthly/woa18_all_o"+t+"_01.nc", decode_times=False)
    DOXY[tt-1,:,:,:] = np.array(data["o_an"][0,:,:,:])
    
    # Nitrate
    data=xr.open_dataset(inpath+"nitrate/monthly/woa18_all_n"+t+"_01.nc", decode_times=False)
    NITRATE[tt-1,:43,:,:] = np.array(data["n_an"][0,:,:,:])
    
    # Phosphate
    data=xr.open_dataset(inpath+"phosphate/monthly/woa18_all_p"+t+"_01.nc", decode_times=False)
    PHOSPHATE[tt-1,:43,:,:] = np.array(data["p_an"][0,:,:,:])
    
# depth to pressure
lat = np.array(data.lat)
lon = np.array(data.lon)

PRES = np.full([len(depth),len(lat),len(lon)],np.nan)

for ii,tmplat in zip(range(len(lat)),lat):
    for jj in range(len(lon)):
        PRES[:,ii,jj]=sw.eos80.pres(depth, [tmplat for i in range(len(depth))])


100%|██████████| 12/12 [00:09<00:00,  1.39it/s]


# Subset

In [12]:
PRES[:,0,0]

array([   0.        ,    5.05695486,   10.11390972,   15.17086506,
         20.22781944,   25.28477478,   30.34173012,   35.39868546,
         40.45563889,   45.51259613,   50.56954956,   55.62650681,
         60.68346024,   65.74041748,   70.81085968,   75.85432434,
         80.92476654,   85.96823883,   91.0386734 ,   96.09563446,
        101.15258789,  126.45084381,  151.74909973,  177.04736328,
        202.34562683,  227.65737915,  252.96911621,  278.28085327,
        303.59262085,  328.91784668,  354.24307251,  379.55480957,
        404.8800354 ,  430.21875   ,  455.54400635,  480.88272095,
        506.22143555,  556.91235352,  607.60327148,  658.30767822,
        709.02557373,  759.75689697,  810.50177002,  861.26013184,
        912.03198242,  962.80377197, 1013.58911133, 1064.38793945,
       1115.21362305, 1166.02600098, 1216.86523438, 1267.70446777,
       1318.57067871, 1369.43688965, 1420.33007812, 1471.20983887,
       1522.13000488])

In [5]:
ds_out = xr.Dataset(coords={'month': np.arange(1,13),
                        'depth': depth,
                        'lon': lon,
                        'lat': lat})
coords = [np.arange(1,13),depth,lat,lon]
dims = ['month','depth','lat','lon']

# add to dataset
ds_out['temp'] = xr.DataArray(TEMP,dims = dims,coords = coords)
ds_out['psal'] = xr.DataArray(PSAL,dims = dims,coords = coords)
ds_out['doxy'] = xr.DataArray(DOXY,dims = dims,coords = coords)
ds_out['nitrate'] = xr.DataArray(NITRATE,dims = dims,coords = coords)
ds_out['phosphate'] = xr.DataArray(PHOSPHATE,dims = dims,coords = coords)
ds_out['pres'] = xr.DataArray(PRES,dims = dims[1:],coords = coords[1:])


# subset
lat_slice = slice(lat_bounds[0], lat_bounds[1])
lon_slice = slice(lon_bounds[0], lon_bounds[1])

ds_out = ds_out.sel(lat = lat_slice, lon = lon_slice)

# Remove any negative concentration values

In [6]:
# set negative oxygen values to nan
ds_out['doxy'] = ds_out.doxy.where(ds_out.doxy >= 0)
ds_out['nitrate'] = ds_out.nitrate.where(ds_out.nitrate >= 0)
ds_out['phosphate'] = ds_out.phosphate.where(ds_out.phosphate >= 0)

# Find Water Column Averages

In [7]:
#-------------- Initialize ------------------------#

n = len(ds_out.month)
m = len(ds_out.lat)
p = len(ds_out.lon)

# mean values to 200 dbar
doxy_40_200 = np.full((n,m,p),np.nan)
temp_40_200 = np.full((n,m,p),np.nan)
psal_40_200 = np.full((n,m,p),np.nan)
nitrate_40_200 = np.full((n,m,p),np.nan)
phosphate_40_200 = np.full((n,m,p),np.nan)

doxy_50_200 = np.full((n,m,p),np.nan)
temp_50_200 = np.full((n,m,p),np.nan)
psal_50_200 = np.full((n,m,p),np.nan)
nitrate_50_200 = np.full((n,m,p),np.nan)
phosphate_50_200 = np.full((n,m,p),np.nan)

doxy_60_200 = np.full((n,m,p),np.nan)
temp_60_200 = np.full((n,m,p),np.nan)
psal_60_200 = np.full((n,m,p),np.nan)
nitrate_60_200 = np.full((n,m,p),np.nan)
phosphate_60_200 = np.full((n,m,p),np.nan)

# annual mean to 200dbar
doxy_40_200_ann_mean = np.full((m,p),np.nan)
temp_40_200_ann_mean = np.full((m,p),np.nan)
psal_40_200_ann_mean = np.full((m,p),np.nan)
nitrate_40_200_ann_mean = np.full((m,p),np.nan)
phosphate_40_200_ann_mean = np.full((m,p),np.nan)

doxy_50_200_ann_mean = np.full((m,p),np.nan)
temp_50_200_ann_mean = np.full((m,p),np.nan)
psal_50_200_ann_mean = np.full((m,p),np.nan)
nitrate_50_200_ann_mean = np.full((m,p),np.nan)
phosphate_50_200_ann_mean = np.full((m,p),np.nan)

doxy_60_200_ann_mean = np.full((m,p),np.nan)
temp_60_200_ann_mean = np.full((m,p),np.nan)
psal_60_200_ann_mean = np.full((m,p),np.nan)
nitrate_60_200_ann_mean = np.full((m,p),np.nan)
phosphate_60_200_ann_mean = np.full((m,p),np.nan)

# mean anomaly to 200dbar
doxy_40_200_anom = np.full((n,m,p),np.nan)
temp_40_200_anom = np.full((n,m,p),np.nan)
psal_40_200_anom = np.full((n,m,p),np.nan)
nitrate_40_200_anom = np.full((n,m,p),np.nan)
phosphate_40_200_anom = np.full((n,m,p),np.nan)

doxy_50_200_anom = np.full((n,m,p),np.nan)
temp_50_200_anom = np.full((n,m,p),np.nan)
psal_50_200_anom = np.full((n,m,p),np.nan)
nitrate_50_200_anom = np.full((n,m,p),np.nan)
phosphate_50_200_anom = np.full((n,m,p),np.nan)

doxy_60_200_anom = np.full((n,m,p),np.nan)
temp_60_200_anom = np.full((n,m,p),np.nan)
psal_60_200_anom = np.full((n,m,p),np.nan)
nitrate_60_200_anom = np.full((n,m,p),np.nan)
phosphate_60_200_anom = np.full((n,m,p),np.nan)

with tqdm(total=n * m * p) as pbar:
    for tt,ii,jj in itertools.product(np.arange(n),np.arange(m),np.arange(p)):
        
        # profiles
        temp = np.array(ds_out.temp[tt,:,ii,jj])
        doxy = np.array(ds_out.doxy[tt,:,ii,jj])
        nitrate = np.array(ds_out.nitrate[tt,:,ii,jj])
        phosphate = np.array(ds_out.phosphate[tt,:,ii,jj])
        psal = np.array(ds_out.psal[tt,:,ii,jj])
        pres = np.array(ds_out.pres[:,ii,jj])
        
        #annual mean profiles
        temp_ann_mean = np.nanmean(ds_out.temp[:,:,ii,jj],0)
        doxy_ann_mean = np.nanmean(ds_out.doxy[:,:,ii,jj],0)
        psal_ann_mean = np.nanmean(ds_out.psal[:,:,ii,jj],0)
        nitrate_ann_mean = np.nanmean(ds_out.nitrate[:,:,ii,jj],0)
        phosphate_ann_mean = np.nanmean(ds_out.phosphate[:,:,ii,jj],0)
        
        # monthly anomaly profiles 
        temp_anom = temp - temp_ann_mean
        doxy_anom = doxy - doxy_ann_mean
        psal_anom = psal - psal_ann_mean
        nitrate_anom = nitrate - nitrate_ann_mean
        phosphate_anom = phosphate - phosphate_ann_mean
                
        # find mean temperature and doxy 40-200dbar
        ind = (pres>= 40) & (pres<= 200)
        
        temp_40_200[tt,ii,jj] = np.nanmean(temp[ind])
        doxy_40_200[tt,ii,jj] = np.nanmean(doxy[ind])
        psal_40_200[tt,ii,jj] = np.nanmean(psal[ind])
        nitrate_40_200[tt,ii,jj] = np.nanmean(nitrate[ind])
        phosphate_40_200[tt,ii,jj] = np.nanmean(phosphate[ind])
        
        temp_40_200_ann_mean[ii,jj] = np.nanmean(temp_ann_mean[ind])
        doxy_40_200_ann_mean[ii,jj] = np.nanmean(doxy_ann_mean[ind])
        psal_40_200_ann_mean[ii,jj] = np.nanmean(psal_ann_mean[ind])
        nitrate_40_200_ann_mean[ii,jj] = np.nanmean(nitrate_ann_mean[ind])
        phosphate_40_200_ann_mean[ii,jj] = np.nanmean(phosphate_ann_mean[ind])
        
        temp_40_200_anom[tt,ii,jj] = np.nanmean(temp_anom[ind])
        doxy_40_200_anom[tt,ii,jj] = np.nanmean(doxy_anom[ind])
        psal_40_200_anom[tt,ii,jj] = np.nanmean(psal_anom[ind])
        nitrate_40_200_anom[tt,ii,jj] = np.nanmean(nitrate_anom[ind])
        phosphate_40_200_anom[tt,ii,jj] = np.nanmean(phosphate_anom[ind])
        
        # find mean temperature and doxy 50-200dbar
        ind = (pres>= 50) & (pres<= 200)
        
        temp_50_200[tt,ii,jj] = np.nanmean(temp[ind])
        doxy_50_200[tt,ii,jj] = np.nanmean(doxy[ind])
        psal_50_200[tt,ii,jj] = np.nanmean(psal[ind])
        nitrate_50_200[tt,ii,jj] = np.nanmean(nitrate[ind])
        phosphate_50_200[tt,ii,jj] = np.nanmean(phosphate[ind])
        
        temp_50_200_ann_mean[ii,jj] = np.nanmean(temp_ann_mean[ind])
        doxy_50_200_ann_mean[ii,jj] = np.nanmean(doxy_ann_mean[ind])
        psal_50_200_ann_mean[ii,jj] = np.nanmean(psal_ann_mean[ind])
        nitrate_50_200_ann_mean[ii,jj] = np.nanmean(nitrate_ann_mean[ind])
        phosphate_50_200_ann_mean[ii,jj] = np.nanmean(phosphate_ann_mean[ind])
        
        temp_50_200_anom[tt,ii,jj] = np.nanmean(temp_anom[ind])
        doxy_50_200_anom[tt,ii,jj] = np.nanmean(doxy_anom[ind])
        psal_50_200_anom[tt,ii,jj] = np.nanmean(psal_anom[ind])
        nitrate_50_200_anom[tt,ii,jj] = np.nanmean(nitrate_anom[ind])
        phosphate_50_200_anom[tt,ii,jj] = np.nanmean(phosphate_anom[ind])

        # find mean temperature and doxy 60-200dbar
        ind = (pres>= 50) & (pres<= 225)
        
        temp_60_200[tt,ii,jj] = np.nanmean(temp[ind])
        doxy_60_200[tt,ii,jj] = np.nanmean(doxy[ind])
        psal_60_200[tt,ii,jj] = np.nanmean(psal[ind])
        nitrate_60_200[tt,ii,jj] = np.nanmean(nitrate[ind])
        phosphate_60_200[tt,ii,jj] = np.nanmean(phosphate[ind])
        
        temp_60_200_ann_mean[ii,jj] = np.nanmean(temp_ann_mean[ind])
        doxy_60_200_ann_mean[ii,jj] = np.nanmean(doxy_ann_mean[ind])
        psal_60_200_ann_mean[ii,jj] = np.nanmean(psal_ann_mean[ind])
        nitrate_60_200_ann_mean[ii,jj] = np.nanmean(nitrate_ann_mean[ind])
        phosphate_60_200_ann_mean[ii,jj] = np.nanmean(phosphate_ann_mean[ind])
        
        temp_60_200_anom[tt,ii,jj] = np.nanmean(temp_anom[ind])
        doxy_60_200_anom[tt,ii,jj] = np.nanmean(doxy_anom[ind])
        psal_60_200_anom[tt,ii,jj] = np.nanmean(psal_anom[ind])
        nitrate_60_200_anom[tt,ii,jj] = np.nanmean(nitrate_anom[ind])
        phosphate_60_200_anom[tt,ii,jj] = np.nanmean(phosphate_anom[ind])
    
        pbar.update(1)

  0%|          | 0/23100 [00:00<?, ?it/s]/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:79: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:80: RuntimeWarning: Mean of empty slice
  0%|          | 41/23100 [00:00<02:56, 130.98it/s]/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:92: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:93: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:94: RuntimeWarning: Mean of empty slice
/hom

In [8]:
# add to dataset

#------------------------------ profiles -------------------------------------#
coords = [ds_out.month,ds_out.lat,ds_out.lon]
dims = ['month','lat', 'lon']

ds_out['temp_40_200'] = xr.DataArray(temp_40_200,dims = dims,coords = coords)
ds_out['doxy_40_200'] = xr.DataArray(doxy_40_200,dims = dims,coords = coords)
ds_out['psal_40_200'] = xr.DataArray(psal_40_200,dims = dims,coords = coords)
ds_out['nitrate_40_200'] = xr.DataArray(nitrate_40_200,dims = dims,coords = coords)
ds_out['phosphate_40_200'] = xr.DataArray(phosphate_40_200,dims = dims,coords = coords)

ds_out['temp_50_200'] = xr.DataArray(temp_50_200,dims = dims,coords = coords)
ds_out['doxy_50_200'] = xr.DataArray(doxy_50_200,dims = dims,coords = coords)
ds_out['psal_50_200'] = xr.DataArray(psal_50_200,dims = dims,coords = coords)
ds_out['nitrate_50_200'] = xr.DataArray(nitrate_50_200,dims = dims,coords = coords)
ds_out['phosphate_50_200'] = xr.DataArray(phosphate_50_200,dims = dims,coords = coords)

ds_out['temp_60_200'] = xr.DataArray(temp_60_200,dims = dims,coords = coords)
ds_out['doxy_60_200'] = xr.DataArray(doxy_60_200,dims = dims,coords = coords)
ds_out['psal_60_200'] = xr.DataArray(psal_60_200,dims = dims,coords = coords)
ds_out['nitrate_60_200'] = xr.DataArray(nitrate_60_200,dims = dims,coords = coords)
ds_out['phosphate_60_200'] = xr.DataArray(phosphate_60_200,dims = dims,coords = coords)

#------------------------------ annual mean -----------------------------------#
coords = [ds_out.lat,ds_out.lon]
dims = ['lat', 'lon']

ds_out['temp_40_200_ann_mean'] = xr.DataArray(temp_40_200_ann_mean,dims = dims,coords = coords)
ds_out['doxy_40_200_ann_mean'] = xr.DataArray(doxy_40_200_ann_mean,dims = dims,coords = coords)
ds_out['psal_40_200_ann_mean'] = xr.DataArray(psal_40_200_ann_mean,dims = dims,coords = coords)
ds_out['nitrate_40_200_ann_mean'] = xr.DataArray(nitrate_40_200_ann_mean,dims = dims,coords = coords)
ds_out['phosphate_40_200_ann_mean'] = xr.DataArray(phosphate_40_200_ann_mean,dims = dims,coords = coords)

ds_out['temp_50_200_ann_mean'] = xr.DataArray(temp_50_200_ann_mean,dims = dims,coords = coords)
ds_out['doxy_50_200_ann_mean'] = xr.DataArray(doxy_50_200_ann_mean,dims = dims,coords = coords)
ds_out['psal_50_200_ann_mean'] = xr.DataArray(psal_50_200_ann_mean,dims = dims,coords = coords)
ds_out['nitrate_50_200_ann_mean'] = xr.DataArray(nitrate_50_200_ann_mean,dims = dims,coords = coords)
ds_out['phosphate_50_200_ann_mean'] = xr.DataArray(phosphate_50_200_ann_mean,dims = dims,coords = coords)

ds_out['temp_60_200_ann_mean'] = xr.DataArray(temp_60_200_ann_mean,dims = dims,coords = coords)
ds_out['doxy_60_200_ann_mean'] = xr.DataArray(doxy_60_200_ann_mean,dims = dims,coords = coords)
ds_out['psal_60_200_ann_mean'] = xr.DataArray(psal_60_200_ann_mean,dims = dims,coords = coords)
ds_out['nitrat_60_200_ann_mean'] = xr.DataArray(nitrate_60_200_ann_mean,dims = dims,coords = coords)
ds_out['phosphate_60_200_ann_mean'] = xr.DataArray(phosphate_60_200_ann_mean,dims = dims,coords = coords)

#---------------------------- monthly anomaly ---------------------------------#
coords = [ds_out.month,ds_out.lat,ds_out.lon]
dims = ['month','lat', 'lon']

ds_out['temp_40_200_anom'] = xr.DataArray(temp_40_200_anom,dims = dims,coords = coords)
ds_out['doxy_40_200_anom'] = xr.DataArray(doxy_40_200_anom,dims = dims,coords = coords)
ds_out['psal_40_200_anom'] = xr.DataArray(psal_40_200_anom,dims = dims,coords = coords)
ds_out['nitrate_40_200_anom'] = xr.DataArray(nitrate_40_200_anom,dims = dims,coords = coords)
ds_out['phosphate_40_200_anom'] = xr.DataArray(phosphate_40_200_anom,dims = dims,coords = coords)

ds_out['temp_50_200_anom'] = xr.DataArray(temp_50_200_anom,dims = dims,coords = coords)
ds_out['doxy_50_200_anom'] = xr.DataArray(doxy_50_200_anom,dims = dims,coords = coords)
ds_out['psal_50_200_anom'] = xr.DataArray(psal_50_200_anom,dims = dims,coords = coords)
ds_out['nitrate_50_200_anom'] = xr.DataArray(nitrate_50_200_anom,dims = dims,coords = coords)
ds_out['phosphate_50_200_anom'] = xr.DataArray(phosphate_50_200_anom,dims = dims,coords = coords)

ds_out['temp_60_200_anom'] = xr.DataArray(temp_60_200_anom,dims = dims,coords = coords)
ds_out['doxy_60_200_anom'] = xr.DataArray(doxy_60_200_anom,dims = dims,coords = coords)
ds_out['psal_60_200_anom'] = xr.DataArray(psal_60_200_anom,dims = dims,coords = coords)
ds_out['nitrate_60_200_anom'] = xr.DataArray(nitrate_60_200_anom,dims = dims,coords = coords)
ds_out['phosphate_60_200_anom'] = xr.DataArray(phosphate_60_200_anom,dims = dims,coords = coords)

ds_out

<xarray.Dataset>
Dimensions:                    (depth: 57, lat: 35, lon: 55, month: 12)
Coordinates:
  * month                      (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * depth                      (depth) float32 0.0 5.0 10.0 ... 1450.0 1500.0
  * lon                        (lon) float32 48.5 49.5 50.5 ... 101.5 102.5
  * lat                        (lat) float32 -1.5 -0.5 0.5 ... 30.5 31.5 32.5
Data variables:
    temp                       (month, depth, lat, lon) float64 27.04 ... nan
    psal                       (month, depth, lat, lon) float64 35.32 ... nan
    doxy                       (month, depth, lat, lon) float64 208.9 ... nan
    nitrate                    (month, depth, lat, lon) float64 0.7416 ... nan
    phosphate                  (month, depth, lat, lon) float64 0.1886 ... nan
    pres                       (depth, lat, lon) float64 0.0 0.0 ... 1.516e+03
    temp_40_200                (month, lat, lon) float64 22.84 22.52 ... nan nan
    doxy_40_200           

# Find Coastal Mask

In [9]:
xx,yy = np.meshgrid(np.array(ds_out.lon),np.array(ds_out.lat))

xx,yy = xx.flatten(), yy.flatten()

mask = mask_coast(xx,yy,[lon_bounds[0],lon_bounds[1],lat_bounds[0],lat_bounds[1]])
mask.shape
mask_xx = xx[mask]
mask_yy = yy[mask]

ds_out['coastal_mask'] = xr.DataArray(mask,dims = ['coastal_loc'],coords =[np.arange(len(mask))])
ds_out['coastal_mask_xx'] = xr.DataArray(mask_xx,dims = ['coastal_pos'],coords =[np.arange(len(mask[mask]))])
ds_out['coastal_mask_yy'] = xr.DataArray(mask_yy,dims = ['coastal_pos'],coords =[np.arange(len(mask[mask]))])
ds_out['coastal_mask_bounds'] = xr.DataArray([lon_bounds[0],lon_bounds[1],lat_bounds[0],lat_bounds[1]],
                                             dims = ['lonmin,lonmax,latmin,latmax'],
                                             coords =[np.arange(4)])


ds_out

<xarray.Dataset>
Dimensions:                      (coastal_loc: 1925, coastal_pos: 439, depth: 57, lat: 35, lon: 55, lonmin,lonmax,latmin,latmax: 4, month: 12)
Coordinates:
  * month                        (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * depth                        (depth) float32 0.0 5.0 10.0 ... 1450.0 1500.0
  * lon                          (lon) float32 48.5 49.5 50.5 ... 101.5 102.5
  * lat                          (lat) float32 -1.5 -0.5 0.5 ... 30.5 31.5 32.5
  * coastal_loc                  (coastal_loc) int64 0 1 2 3 ... 1922 1923 1924
  * coastal_pos                  (coastal_pos) int64 0 1 2 3 ... 435 436 437 438
  * lonmin,lonmax,latmin,latmax  (lonmin,lonmax,latmin,latmax) int64 0 1 2 3
Data variables:
    temp                         (month, depth, lat, lon) float64 27.04 ... nan
    psal                         (month, depth, lat, lon) float64 35.32 ... nan
    doxy                         (month, depth, lat, lon) float64 208.9 ... nan
    nitrate          

# SAVE

In [10]:
# delete if already present
if os.path.isfile(outfn):
    os.remove(outfn)

ds_out.to_netcdf(outfn,mode='w',format = "NETCDF4")